# Collaborative filtering recommender system

## Dataset description

In this assignment you will use Apache Spark to build simple movie recommendation system on big MovieLens 20M dataset (https://grouplens.org/datasets/movielens/). The dataset contains 20 million ratings for 27000 movies given by 138000 users. 

Dataset format: comma-separated values (https://en.wikipedia.org/wiki/CSV)

The first line is a header:
userId,movieId,rating,timestamp

_userId_, _movieId_ are integers representing user and movies identifiers correspondingly

_rating_ is a floating point number from 1.0 to 5.0

_timestamp_ is an integer but it won’t be used in the assignment :)

## Reading dataset

Init spark session

In [1]:
from pyspark.sql import SparkSession

spark_session = SparkSession.builder \
    .enableHiveSupport() \
    .appName("recSys")\
    .master("local[8]")\
    .getOrCreate()
sc = spark_session.sparkContext

In [2]:
# You can now use spark session or spark context to read csv.
data = spark_session.read.option("inferschema", "true").csv("/data/movielens/ratings_100k.csv", header="true").drop("timestamp")

In this assignment you will use collaborative filtering approach for making predictions. To find latent vector representation for users and items we suggest you to use explicit ALS method. Refer to the Spark MLLib documentation for Python API details: https://spark.apache.org/docs/2.1.0/api/python/pyspark.mllib.html

Split dataset into three folds: on each iteration one fold will be used for testing and the other two folds will be used for training. Wrap necessary values with _Rating_ class.

In [3]:
from pyspark.mllib.recommendation import Rating

fold_count = 3

In [4]:
fold1, fold2,fold3 = data.randomSplit([0.33, 0.33, 0.33], seed = 123)
folds_init = [fold1, fold2,fold3]
folds = []
for idx in range(len(folds_init)):
    trains=folds_init[:idx] + folds_init[idx+1:]
    train = trains[0]
    for t in trains[1:]:
        train = train.union(t)
    test=folds_init[idx]
    folds.append((train,test))

Recommendations: to make your solution more efficient prepare RDDs for all the folds beforehand running the training process.

## Training ALS model

Now it is time to choose training parameters. It is recommended to set the rank equal to 20,
regularization term lambda equal to 0.01 and the number of iterations equal to 5.
Feel free to experiment and choose your own parameters to see how they influence the final score.

In [66]:
rank = 20
iterations = 10
lambda_ = 0.151

Train the explicit ALS model on two of three folds and evaluate its performance on remaining test fold. Performance evaluation should be done using the classic RMSE metric.

In [6]:
scores = []

In [7]:
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator

In [67]:
fold_count = 3
for test_fold_index in range(fold_count):
    train,test=folds[test_fold_index]
    #userCol="userId", itemCol="movieId", ratingCol="rating", coldStartStrategy="drop", nonnegative=True
    als = ALS(userCol="userId", itemCol="movieId", ratingCol="rating",rank=rank,maxIter=iterations,regParam=lambda_)
    #print("training")
    model = als.fit(train)
    #print("prediction")
    predictions = model.transform(test)
    evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating",predictionCol="prediction")
    rmse = evaluator.evaluate(predictions.dropna())
    scores.append(rmse)
    #print("Root-mean-square error = " + str(rmse))

training
prediction
Root-mean-square error = 0.9202573824933572
training
prediction
Root-mean-square error = 0.9107565064758453
training
prediction
Root-mean-square error = 0.9129893163840864


Average RMSE scores on all the three folds and output the result to stdout.
You could refer to publicly available benchmarks (e.g. http://mymedialite.net/examples/datasets.html)
to find out what score to expect.

In [68]:
score = sum(scores) / len(scores)

In [69]:
score

0.9146677351177629